# LQR appy to a Robot Arm in MuJoCo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mujoco
import mujoco.viewer as viewer
import mediapy as media
import scipy

## Load Model

In [2]:
# load model
xml = "../mujoco_menagerie/universal_robots_ur5e/scene.xml"
model = mujoco.MjModel.from_xml_path(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

## Render random noise

In [3]:
DURATION  = 5   # seconds
FRAMERATE = 60  # Hz

# Make a new camera, move it to a closer distance.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 1

# Initialize the robot pose.
mujoco.mj_resetDataKeyframe(model, data, 1)

frames = []
while data.time < DURATION:
    # Set control vector.
    data.ctrl = np.random.randn(model.nu)
    
    # Step the simulation.
    mujoco.mj_step(model, data)    

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        # # Set the lookat point to the humanoid's center of mass.
        # camera.lookat = data.body('torso').subtree_com
        
        renderer.update_scene(data, camera)
        pixels = renderer.render()
        frames.append(pixels)

# Display video.
media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


## Define setpoint for end effector position

In [14]:
#calculate the torques to mantain the end effector position
mujoco.mj_forward(model, data)
data.qacc = 0
qpos0 = [3*pi/2, -pi/2, pi/2, 3*pi/2, 3*pi/2, 0] # Save the position setpoint.
mujoco.mj_inverse(model, data) # compute actuator force
qfrc0 = data.qfrc_inverse.copy() # save actuactor forces for the position
print('desired forces:', qfrc0)

ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(data.actuator_moment)
ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
print('control setpoint:', ctrl0)

desired forces: [-1.92503487e+01 -1.49294699e+02 -1.50545489e+02 -2.76251900e+01
 -5.75457956e+00 -5.91187414e-09]
control setpoint: [-1.92503487e+01 -1.49294699e+02 -1.50545489e+02 -2.76251900e+01
 -5.75457956e+00 -5.91187414e-09]
position [4.71238898038469, -1.5707963267948966, 1.5707963267948966, 4.71238898038469, 4.71238898038469, 0]


In [20]:
DURATION  = 2   # seconds
FRAMERATE = 60  # Hz

# Make a new camera position #TODO:update this position to one better.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2
camera.azimuth = -25

# Set the state and controls to their setpoints.
mujoco.mj_resetData(model, data)
data.qpos = qpos0
data.ctrl = ctrl0

frames = []
while data.time < DURATION:
  # Step the simulation.
    mujoco.mj_step(model, data)

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        # Set the lookat point to the humanoid's center of mass.
        # camera.lookat = data.body('torso').subtree_com
        renderer.update_scene(data, camera)
        pixels = renderer.render()
        frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


## Design LQR controller

In [47]:
nu = model.nu  # Alias for the number of actuators.
R = np.eye(nu)
print(R)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [46]:
# Get all joint names.
joint_names = [model.joint(i).name for i in range(model.njnt)]

# Get indices of joints.
nv = model.nv
root_dofs = range(6)

all_dofs = [
    model.joint(name).dofadr[0]
    for name in joint_names
]

# print(root_dofs)
# Cost coefficients
COST = 10

# Construct the Q matrix
Qjoint = np.eye(nv)
# print(Qjoint)

# Qjoint[all_dofs] *= COST

Qpos = COST * Qjoint
# print(Qjoint)
# print(Qpos)

Q = np.block([[Qpos, np.zeros((nv, nv))],
              [np.zeros((nv, 2*nv))]])
print(Q)

[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


## Compute gain K

In [40]:
nu = model.nu # number of actuators
nv = model.nv  # number of degree of freedoms.

# set initial state and control
mujoco.mj_resetData(model, data)
data.ctrl = ctrl0
data.qpos = qpos0

# define the continuous-time system matrices A, B, C
A = np.zeros((2*nv, 2*nv))
B = np.zeros((2*nv, nu))

epsilon = 1e-6
flg_centered = False

mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, None, None)

# Solve discrete Riccati equation.
P = scipy.linalg.solve_discrete_are(A, B, Q, R)

# Compute the feedback gain matrix K.
K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

print(K)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.

## Render Test

In [41]:
# Parameters.
DURATION = 5         # seconds
FRAMERATE = 60        # Hz

# Reset data, set initial pose.
mujoco.mj_resetData(model, data)
data.qpos = qpos0

# Allocate position difference dq.
dq = np.zeros(model.nv)

# Make a new camera position #TODO:update this position to one better.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2
camera.azimuth = -25


frames = []
while data.time < DURATION:
    # Get state difference dx.
    mujoco.mj_differentiatePos(model, dq, 1, qpos0, data.qpos)
    dx = np.hstack((dq, data.qvel)).T

    # LQR control law.
    data.ctrl = ctrl0 - K @ dx

    # Step the simulation.
    mujoco.mj_step(model, data)

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.
